In [1]:
import glob
import os
import datetime
import string

import pandas as pd
import numpy as np

import sqlalchemy

Imported and concat to create data frame around the bikes.  I used glob to streamline the addition of the various csv files with similar names. 

In [2]:
df_list = []
bike_files = glob.glob('bike-rental-starter-kit/data/JC-*-citibike-tripdata.csv')
for file in bike_files:
    file = pd.read_csv(file)
    df_list.append(file)

df_bike = pd.concat(df_list)
df_bike['id'] = df_bike.index
df_bike.reset_index(drop=True, inplace=True)
df_bike.head()

,Trip Duration,Start Time,Stop Time,Start Station ID,Start Station Name,Start Station Latitude,Start Station Longitude,End Station ID,End Station Name,End Station Latitude,End Station Longitude,Bike ID,User Type,Birth Year,Gender,id
0,361,2016-02-01 00:31:18,2016-02-01 00:37:19,3202,Newport PATH,40.727224,-74.033759,3203,Hamilton Park,40.727596,-74.044247,24393,Subscriber,1975.0,1,0
1,297,2016-02-01 01:55:05,2016-02-01 02:00:02,3195,Sip Ave,40.730743,-74.063784,3194,McGinley Square,40.725340,-74.067622,24394,Subscriber,1985.0,2,1
2,1155,2016-02-01 02:40:05,2016-02-01 02:59:20,3183,Exchange Place,40.716247,-74.033459,3210,Pershing Field,40.742677,-74.051789,24676,Subscriber,1976.0,1,2
3,1769,2016-02-01 05:11:28,2016-02-01 05:40:58,3214,Essex Light Rail,40.712774,-74.036486,3203,Hamilton Park,40.727596,-74.044247,24700,Subscriber,1974.0,2,3
4,935,2016-02-01 05:48:24,2016-02-01 06:03:59,3203,Hamilton Park,40.727596,-74.044247,3214,Essex Light Rail,40.712774,-74.036486,24639,Subscriber,1974.0,2,4


Theres nothing of note in the head. I need to find out how many total rows there are. It looks like Gender is quantified as 1 or 2 rather than male vs female.  Theres no way to note which is which. 

In [3]:
df_bike.tail()

,Trip Duration,Start Time,Stop Time,Start Station ID,Start Station Name,Start Station Latitude,Start Station Longitude,End Station ID,End Station Name,End Station Latitude,End Station Longitude,Bike ID,User Type,Birth Year,Gender,id
247579,249,2016-05-31 23:16:00,2016-05-31 23:20:10,3186,Grove St PATH,40.719586,-74.043117,3209,Brunswick St,40.724176,-74.050656,24668,Subscriber,1976.0,1,19483
247580,650,2016-05-31 23:18:32,2016-05-31 23:29:23,3209,Brunswick St,40.724176,-74.050656,3211,Newark Ave,40.721525,-74.046305,24517,Subscriber,1986.0,1,19484
247581,2048,2016-05-31 23:25:28,2016-05-31 23:59:36,3199,Newport Pkwy,40.728745,-74.032108,3199,Newport Pkwy,40.728745,-74.032108,24573,Subscriber,1975.0,1,19485
247582,455,2016-05-31 23:31:57,2016-05-31 23:39:32,3220,5 Corners Library,40.734961,-74.059503,3215,Central Ave,40.746730,-74.049251,24561,Subscriber,1964.0,1,19486
247583,239,2016-05-31 23:47:38,2016-05-31 23:51:37,3185,City Hall,40.717732,-74.043845,3211,Newark Ave,40.721525,-74.046305,24685,Subscriber,1993.0,1,19487


Total of 247,583 entries

In [4]:
df_bike.isna().sum()

Trip Duration                  0
Start Time                     0
Stop Time                      0
Start Station ID               0
Start Station Name             0
Start Station Latitude         0
Start Station Longitude        0
End Station ID                 0
End Station Name               0
End Station Latitude           0
End Station Longitude          0
Bike ID                        0
User Type                    380
Birth Year                 18999
Gender                         0
id                             0
dtype: int64

Birth year is missing a large chunk of data. Is that purposeful?

In [5]:
df_bike.describe()

,Trip Duration,Start Station ID,Start Station Latitude,Start Station Longitude,End Station ID,End Station Latitude,End Station Longitude,Bike ID,Birth Year,Gender,id
count,2.475840e+05,247584.000000,247584.000000,247584.000000,247584.000000,247584.000000,247584.000000,247584.000000,228585.000000,247584.000000,247584.000000
mean,8.856305e+02,3207.065206,40.723121,-74.046438,3203.572553,40.722594,-74.045855,24935.260481,1979.335276,1.123534,12094.850350
std,3.593798e+04,26.955103,0.008199,0.011211,61.579494,0.007958,0.011283,748.469712,9.596809,0.518687,8356.014112
min,6.100000e+01,3183.000000,40.692640,-74.096937,147.000000,40.692216,-74.096937,14552.000000,1900.000000,0.000000,0.000000
25%,2.480000e+02,3186.000000,40.717732,-74.050656,3186.000000,40.716540,-74.050444,24491.000000,1974.000000,1.000000,5157.750000
50%,3.900000e+02,3201.000000,40.721525,-74.044247,3199.000000,40.721124,-74.043117,24609.000000,1981.000000,1.000000,10806.000000
75%,6.660000e+02,3211.000000,40.727596,-74.038051,3211.000000,40.727224,-74.036486,24719.000000,1986.000000,1.000000,17855.250000
max,1.632981e+07,3426.000000,40.752559,-74.032108,3426.000000,40.801343,-73.957390,27274.000000,2000.000000,2.000000,34148.000000


The birth years are confusing.  The earliest year is 1900 which means they are about 116. The majority of rentals by gender are 1.  The trip duration is confusing here. I need to separate them into hours and minutes 

In [6]:
df_bike.dtypes

Trip Duration                int64
Start Time                  object
Stop Time                   object
Start Station ID             int64
Start Station Name          object
Start Station Latitude     float64
Start Station Longitude    float64
End Station ID               int64
End Station Name            object
End Station Latitude       float64
End Station Longitude      float64
Bike ID                      int64
User Type                   object
Birth Year                 float64
Gender                       int64
id                           int64
dtype: object

The objects for start and stop time need to be fixed.  They should be datetime format.

In [7]:
df_bike['Start Time'] = pd.to_datetime(df_bike['Start Time'], format='%Y-%m-%d %H:%M:%S')
df_bike['Stop Time'] = pd.to_datetime(df_bike['Stop Time'], format='%Y-%m-%d %H:%M:%S')

In [8]:
df_bike.dtypes

Trip Duration                       int64
Start Time                 datetime64[ns]
Stop Time                  datetime64[ns]
Start Station ID                    int64
Start Station Name                 object
Start Station Latitude            float64
Start Station Longitude           float64
End Station ID                      int64
End Station Name                   object
End Station Latitude              float64
End Station Longitude             float64
Bike ID                             int64
User Type                          object
Birth Year                        float64
Gender                              int64
id                                  int64
dtype: object

#### I need to keep looking at the following columns:
    Birth Year: minimum is small and a large chunk of data is missing
    User Type: missing data
    Trip Duration: large maximum
    Gender: should only have 2 options (1 and 2) but has some 0s
  
Before I do this I want to remove any white space in the column names

In [9]:
df_bike.columns = [x.replace(' ', '_').lower() for x in df_bike.columns]

In [10]:
df_bike.columns

Index(['trip_duration', 'start_time', 'stop_time', 'start_station_id',
       'start_station_name', 'start_station_latitude',
       'start_station_longitude', 'end_station_id', 'end_station_name',
       'end_station_latitude', 'end_station_longitude', 'bike_id', 'user_type',
       'birth_year', 'gender', 'id'],
      dtype='object')

## Birth Year
Calculating age will help investigate the outlier and might make it easier to use later

In [11]:
df_bike['age'] = 2016 - df_bike['birth_year'].values

In [12]:
df_bike['age'].sort_values(ascending = False)

186245    116.0
31008      82.0
72509      79.0
184757     79.0
173695     79.0
          ...  
247463      NaN
247464      NaN
247534      NaN
247537      NaN
247553      NaN
Name: age, Length: 247584, dtype: float64

The 116 is what I predicted. I need to go back to the NaN values.  I'm going to sort out show just the highest 15 values.

In [13]:
df_bike['age'].sort_values(ascending = False)[:10]

186245    116.0
31008      82.0
72509      79.0
184757     79.0
173695     79.0
157596     79.0
226979     76.0
143125     76.0
170642     76.0
148238     75.0
Name: age, dtype: float64

116 is an outlier and should be removed as to not impact the statistical analysis. 

In [14]:
df_bike.drop([186245], inplace = True)

In [15]:
df_bike['age'].sort_values(ascending = False)[:10]

31008     82.0
72509     79.0
184757    79.0
157596    79.0
173695    79.0
170642    76.0
143125    76.0
226979    76.0
79514     75.0
221808    75.0
Name: age, dtype: float64

### Missing Birth Years
I am going to define the missing birth years

In [16]:
missing_birth_years = df_bike[df_bike[['birth_year']].isnull().any(axis=1)]
missing_birth_years.head()

,trip_duration,start_time,stop_time,start_station_id,start_station_name,start_station_latitude,start_station_longitude,end_station_id,end_station_name,end_station_latitude,end_station_longitude,bike_id,user_type,birth_year,gender,id,age
125,3665,2016-02-01 09:56:46,2016-02-01 10:57:52,3212,Christ Hospital,40.734786,-74.050444,3185,City Hall,40.717732,-74.043845,24531,Customer,NaN,0,125,NaN
148,1081,2016-02-01 11:43:51,2016-02-01 12:01:53,3183,Exchange Place,40.716247,-74.033459,3192,Liberty Light Rail,40.711242,-74.055701,24624,Customer,NaN,0,148,NaN
154,1100,2016-02-01 12:04:35,2016-02-01 12:22:56,3192,Liberty Light Rail,40.711242,-74.055701,3192,Liberty Light Rail,40.711242,-74.055701,24624,Customer,NaN,0,154,NaN
163,1579,2016-02-01 12:26:09,2016-02-01 12:52:29,3192,Liberty Light Rail,40.711242,-74.055701,3183,Exchange Place,40.716247,-74.033459,24624,Customer,NaN,0,163,NaN
168,1619,2016-02-01 12:53:44,2016-02-01 13:20:44,3186,Grove St PATH,40.719586,-74.043117,3203,Hamilton Park,40.727596,-74.044247,24510,Customer,NaN,0,168,NaN


In [17]:
missing_birth_years.tail()

,trip_duration,start_time,stop_time,start_station_id,start_station_name,start_station_latitude,start_station_longitude,end_station_id,end_station_name,end_station_latitude,end_station_longitude,bike_id,user_type,birth_year,gender,id,age
247463,1753,2016-05-31 20:02:01,2016-05-31 20:31:14,3213,Van Vorst Park,40.718489,-74.047727,3184,Paulus Hook,40.714145,-74.033552,24669,Customer,NaN,0,19367,NaN
247464,1706,2016-05-31 20:03:06,2016-05-31 20:31:32,3213,Van Vorst Park,40.718489,-74.047727,3184,Paulus Hook,40.714145,-74.033552,24719,Customer,NaN,0,19368,NaN
247534,1305,2016-05-31 21:11:03,2016-05-31 21:32:48,3187,Warren St,40.721124,-74.038051,3187,Warren St,40.721124,-74.038051,24721,Customer,NaN,0,19438,NaN
247537,648,2016-05-31 21:19:44,2016-05-31 21:30:33,3187,Warren St,40.721124,-74.038051,3187,Warren St,40.721124,-74.038051,24399,Customer,NaN,0,19441,NaN
247553,267,2016-05-31 21:42:02,2016-05-31 21:46:30,3183,Exchange Place,40.716247,-74.033459,3185,City Hall,40.717732,-74.043845,24685,Customer,NaN,0,19457,NaN


When I saw the first 5 I thought it was going to be on one day. But it looks like it's throughout all the duration of months. 
The first 5 are all customers and all have an unknown gender
That holds up in the last 5 as well. I'm going to run another test to check once more if the pattern is true. 

In [18]:
missing_birth_years[['user_type', 'gender']].value_counts(dropna=False)

user_type   gender
Customer    0         15470
Subscriber  0          3529
dtype: int64

All users with missing birth years have an uknown gender.
Most users with a missing birth year are customers but about 1/6 of them are subscribers

In order to check if I can delete missing birth years I need to see what percentage user types they are.

In [19]:
missing_birth_years['user_type'].value_counts() / df_bike['user_type'].value_counts() 

Customer      0.996714
Subscriber    0.015232
Name: user_type, dtype: float64

I can't delete the unknown birth years because it's almost 100% of customers.  This also means most customers gender is going to be unknown

## User Type
There is a small portion of user types that are unknown.  I should check to see if they are also missing a gender or a birth year

In [20]:
missing_user_type = df_bike[df_bike[['user_type']].isnull().any(axis=1)]
missing_user_type.head()

,trip_duration,start_time,stop_time,start_station_id,start_station_name,start_station_latitude,start_station_longitude,end_station_id,end_station_name,end_station_latitude,end_station_longitude,bike_id,user_type,birth_year,gender,id,age
8546,943,2016-08-01 09:48:21,2016-08-01 10:04:05,3214,Essex Light Rail,40.712774,-74.036486,3199,Newport Pkwy,40.728745,-74.032108,24609,NaN,1987.0,1,296,29.0
8634,2319,2016-08-01 12:29:23,2016-08-01 13:08:03,3199,Newport Pkwy,40.728745,-74.032108,3184,Paulus Hook,40.714145,-74.033552,24609,NaN,1987.0,1,384,29.0
10848,365,2016-08-03 10:46:37,2016-08-03 10:52:43,3214,Essex Light Rail,40.712774,-74.036486,3202,Newport PATH,40.727224,-74.033759,24724,NaN,1987.0,1,2598,29.0
13241,396,2016-08-05 10:20:57,2016-08-05 10:27:34,3214,Essex Light Rail,40.712774,-74.036486,3202,Newport PATH,40.727224,-74.033759,24530,NaN,1987.0,1,4991,29.0
13385,581,2016-08-05 14:06:10,2016-08-05 14:15:51,3202,Newport PATH,40.727224,-74.033759,3183,Exchange Place,40.716247,-74.033459,24668,NaN,1987.0,1,5135,29.0


The first 5 all have a gender of 1 and an age of 29(birth year of 1987).
I need to check if the pattern is true for all of the data.

In [21]:
missing_user_type[['gender', 'age']].value_counts(dropna=False)

gender  age 
1       29.0    89
2       28.0    20
1       25.0    18
2       26.0    14
1       24.0    11
        26.0    11
        33.0    10
2       25.0    10
1       28.0    10
2       34.0     8
        31.0     8
1       32.0     7
2       35.0     7
1       36.0     7
        37.0     7
2       24.0     7
        30.0     7
1       34.0     6
        27.0     6
        43.0     6
        35.0     5
        40.0     5
2       29.0     5
        23.0     5
1       46.0     5
        58.0     5
2       27.0     4
1       53.0     4
        45.0     4
2       32.0     4
1       21.0     4
        49.0     4
        31.0     4
        38.0     4
2       46.0     4
        54.0     3
1       23.0     3
        19.0     3
        47.0     3
        69.0     3
        52.0     3
        56.0     3
        55.0     3
2       41.0     3
        38.0     2
1       18.0     2
        57.0     2
        44.0     2
2       72.0     2
        21.0     1
        36.0     1
        37.0     1

The pattern isn't true accross all the data.  I need to just keep this in mind as I go through the analysis. 

## Trip Duration
Trip duration maximum was suspisiously large. I need to make the trip time easier to interpret.

In [22]:
df_bike['trip_minutes'] = round(df_bike['trip_duration'] / 60, 2)
df_bike['trip_hours'] = round(df_bike['trip_duration'] / (60**2), 2)
df_bike[['trip_minutes', 'trip_hours', 'trip_duration']].describe()

,trip_minutes,trip_hours,trip_duration
count,247583.000000,247583.000000,2.475830e+05
mean,14.760494,0.246006,8.856300e+02
std,598.967493,9.982797,3.593805e+04
min,1.020000,0.020000,6.100000e+01
25%,4.130000,0.070000,2.480000e+02
50%,6.500000,0.110000,3.900000e+02
75%,11.100000,0.180000,6.660000e+02
max,272163.470000,4536.060000,1.632981e+07


The shortest trip length was just over one minute. When I looked at the data dictionary it said that trips that were less than 60 seconds were assumed to be false starts/stops and were pruned. 

The longest trip was 4536 hours which is well above policy (24 hours). I am going to sort in descending order to see if theres a lot of these overly large hours. 

In [23]:
df_bike['trip_hours'].sort_values(ascending=False)

136316    4536.06
200338    1340.80
3233       584.48
3269       583.49
3371       575.34
           ...   
75123        0.02
85349        0.02
207794       0.02
38879        0.02
54550        0.02
Name: trip_hours, Length: 247583, dtype: float64

It looks like there is a decent amount of trips that were more than 24 hours. I am going to leave them in the data for now because it could be an issue with docking the bike. 

# Weather Data

In [24]:
df_weather = pd.read_csv('bike-rental-starter-kit/data/newark_airport_2016.csv')
df_weather.head()

,STATION,NAME,DATE,AWND,PGTM,PRCP,SNOW,SNWD,TAVG,TMAX,TMIN,TSUN,WDF2,WDF5,WSF2,WSF5
0,USW00014734,"NEWARK LIBERTY INTERNATIONAL AIRPORT, NJ US",2016-01-01,12.75,NaN,0.0,0.0,0.0,41,43,34,NaN,270,280.0,25.9,35.1
1,USW00014734,"NEWARK LIBERTY INTERNATIONAL AIRPORT, NJ US",2016-01-02,9.40,NaN,0.0,0.0,0.0,36,42,30,NaN,260,260.0,21.0,25.1
2,USW00014734,"NEWARK LIBERTY INTERNATIONAL AIRPORT, NJ US",2016-01-03,10.29,NaN,0.0,0.0,0.0,37,47,28,NaN,270,250.0,23.9,30.0
3,USW00014734,"NEWARK LIBERTY INTERNATIONAL AIRPORT, NJ US",2016-01-04,17.22,NaN,0.0,0.0,0.0,32,35,14,NaN,330,330.0,25.9,33.1
4,USW00014734,"NEWARK LIBERTY INTERNATIONAL AIRPORT, NJ US",2016-01-05,9.84,NaN,0.0,0.0,0.0,19,31,10,NaN,360,350.0,25.1,31.1


In [25]:
df_weather.describe()

,AWND,PGTM,PRCP,SNOW,SNWD,TAVG,TMAX,TMIN,TSUN,WDF2,WDF5,WSF2,WSF5
count,366.000000,0.0,366.000000,366.000000,366.000000,366.000000,366.000000,366.000000,0.0,366.000000,364.000000,366.000000,364.000000
mean,9.429973,NaN,0.104945,0.098087,0.342623,57.196721,65.991803,48.459016,NaN,217.841530,228.269231,20.484426,26.801648
std,3.748174,NaN,0.307496,1.276498,2.078510,17.466981,18.606301,17.135790,NaN,102.548282,97.415777,6.848390,8.882610
min,2.460000,NaN,0.000000,0.000000,0.000000,8.000000,18.000000,0.000000,NaN,10.000000,10.000000,6.900000,10.100000
25%,6.765000,NaN,0.000000,0.000000,0.000000,43.000000,51.250000,35.000000,NaN,150.000000,150.000000,15.000000,19.900000
50%,8.720000,NaN,0.000000,0.000000,0.000000,56.000000,66.000000,47.000000,NaN,240.000000,260.000000,19.900000,25.100000
75%,11.410000,NaN,0.030000,0.000000,0.000000,74.000000,83.000000,64.000000,NaN,300.000000,300.000000,23.900000,31.100000
max,22.820000,NaN,2.790000,24.000000,20.100000,89.000000,99.000000,80.000000,NaN,360.000000,360.000000,48.100000,66.000000


In [26]:
df_weather.dtypes

STATION     object
NAME        object
DATE        object
AWND       float64
PGTM       float64
PRCP       float64
SNOW       float64
SNWD       float64
TAVG         int64
TMAX         int64
TMIN         int64
TSUN       float64
WDF2         int64
WDF5       float64
WSF2       float64
WSF5       float64
dtype: object

PGTM and TSUN are completely missing, they will be dropped. WDFx and WSFx refer to the fastest x-minute wind, and are difficult to connect directly to our data (since we don't know when that speed occurred.) Instead we'll use AWND as the wind-speed variable.

We don't need the station information since it is all currently the same.

In [27]:
df_weather.drop(['STATION','NAME','PGTM','TSUN', 'WDF2', 'WDF5', 'WSF2', 'WSF5'],axis=1, inplace=True)

In [28]:
df_weather.dtypes

DATE     object
AWND    float64
PRCP    float64
SNOW    float64
SNWD    float64
TAVG      int64
TMAX      int64
TMIN      int64
dtype: object

I want the date to be a DateTime rather than an object

In [30]:
df_weather['DATE'] = pd.to_datetime(df_weather['DATE'],yearfirst=True)

In [31]:
df_weather.dtypes

DATE    datetime64[ns]
AWND           float64
PRCP           float64
SNOW           float64
SNWD           float64
TAVG             int64
TMAX             int64
TMIN             int64
dtype: object

In [32]:
df_weather.describe()

,AWND,PRCP,SNOW,SNWD,TAVG,TMAX,TMIN
count,366.000000,366.000000,366.000000,366.000000,366.000000,366.000000,366.000000
mean,9.429973,0.104945,0.098087,0.342623,57.196721,65.991803,48.459016
std,3.748174,0.307496,1.276498,2.078510,17.466981,18.606301,17.135790
min,2.460000,0.000000,0.000000,0.000000,8.000000,18.000000,0.000000
25%,6.765000,0.000000,0.000000,0.000000,43.000000,51.250000,35.000000
50%,8.720000,0.000000,0.000000,0.000000,56.000000,66.000000,47.000000
75%,11.410000,0.030000,0.000000,0.000000,74.000000,83.000000,64.000000
max,22.820000,2.790000,24.000000,20.100000,89.000000,99.000000,80.000000


In [33]:
df_weather.columns = ['rec_date','avg_wind','prcp','snow_amt','snow_depth','tavg','tmax','tmin']

In [34]:
df_weather['rain'] = df_weather['prcp'].apply(lambda x: 1 if x > 0 else 0).astype(bool)
df_weather['snow'] = df_weather['snow_amt'].apply(lambda x: 1 if x > 0 else 0).astype(bool)

In [35]:
df_weather.columns

Index(['rec_date', 'avg_wind', 'prcp', 'snow_amt', 'snow_depth', 'tavg',
       'tmax', 'tmin', 'rain', 'snow'],
      dtype='object')

In [39]:
df_weather.dtypes

rec_date      datetime64[ns]
avg_wind             float64
prcp                 float64
snow_amt             float64
snow_depth           float64
tavg                   int64
tmax                   int64
tmin                   int64
rain                    bool
snow                    bool
dtype: object

In [38]:
df_bike.dtypes

trip_duration                       int64
start_time                 datetime64[ns]
stop_time                  datetime64[ns]
start_station_id                    int64
start_station_name                 object
start_station_latitude            float64
start_station_longitude           float64
end_station_id                      int64
end_station_name                   object
end_station_latitude              float64
end_station_longitude             float64
bike_id                             int64
user_type                          object
birth_year                        float64
gender                              int64
id                                  int64
age                               float64
trip_minutes                      float64
trip_hours                        float64
dtype: object

# Create a Schema
df_weather can be its own table

df_bikes can be broken up into its own tables

Trip info = trip_duration, start_time, stop_time, trip_minutes, trip_hours, bike_id

Station info = start_station_id/name start station lat/long, endstation id/name, end station lat/long

User info = user type, birth year, age, gender
 
Corresponding columns will be (trip)start_time and (weather)rec_date the others will be connected through id